In [1]:
#%matplotlib widget
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import pack_vis
import pack_dimer
import pack_vis_sol
import pack_cost
import numpy as np
import cupy as cp
import copy
kgs.set_float32(True)

local


In [2]:
import glob

sols_controller = []
for N_trees in np.arange(1,201):
    gl=glob.glob(kgs.temp_dir + f'/*runs/done/ga_N{N_trees}_*.pickle') + glob.glob(kgs.temp_dir + f'/*runs/extra/ga_N{N_trees}_*.pickle') + glob.glob(kgs.temp_dir + f'/*runs/repro/ga_N{N_trees}_*.pickle') + \
        glob.glob(kgs.temp_dir + f'/rerun_done/done/ga_N{N_trees}_*.pickle')
    best_res = None
    best_score = np.inf
    for g in gl:        
        rr = kgs.dill_load(g)
        if not (rr.ga.ga_base.initializer.ref_sol_crystal_type == 'Perfect dimer'):
            print(g, 'skipped')
            continue
        score = rr.ga.champions[0].fitness[0,0]
        if score < best_score:
            best_score = score
            best_res = rr
    if best_res is None:
        sols_controller.append(None)
    else:
        sols_controller.append(best_res.ga.champions[0].phenotype)
    #sols_controller.append(rr.ga.champions[0].phenotype)
    #res_per_tree[N_trees].append([x[0] for x in rr.ga.best_costs_per_generation[-1]])


/mnt/d//packing/temp/controller_runs/done/ga_N1_seed84_done.pickle skipped
/mnt/d//packing/temp/controller_runs/done/ga_N1_seed86_done.pickle skipped
/mnt/d//packing/temp/controller_runs/done/ga_N2_seed84_done.pickle skipped
/mnt/d//packing/temp/controller_runs/done/ga_N2_seed86_done.pickle skipped
/mnt/d//packing/temp/controller_runs/done/ga_N3_seed84_done.pickle skipped
/mnt/d//packing/temp/controller_runs/done/ga_N3_seed86_done.pickle skipped
/mnt/d//packing/temp/controller_runs/done/ga_N4_seed84_done.pickle skipped
/mnt/d//packing/temp/controller_runs/done/ga_N4_seed86_done.pickle skipped
/mnt/d//packing/temp/controller_runs/done/ga_N4_seed88_done.pickle skipped
/mnt/d//packing/temp/controller_runs/done/ga_N5_seed84_done.pickle skipped
/mnt/d//packing/temp/controller_runs/done/ga_N5_seed86_done.pickle skipped
/mnt/d//packing/temp/controller_runs/done/ga_N6_seed84_done.pickle skipped
/mnt/d//packing/temp/controller_runs/done/ga_N6_seed86_done.pickle skipped
/mnt/d//packing/temp/cont

In [3]:
import matplotlib.pyplot as plt
def circular_mean_mod_pi(angles):
    """Compute mean direction on periodic domain [0, π)"""
    # Use 2*angle to map π-periodicity to 2π-periodicity
    sin_sum = np.sin(2*angles).mean()
    cos_sum = np.cos(2*angles).mean()
    mean_angle = np.arctan2(sin_sum, cos_sum) / 2
    return mean_angle % np.pi

def circular_median_mod_pi(angles):
    """Find angle that minimizes sum of circular distances on [0, π)"""
    from scipy.optimize import minimize_scalar
    
    def circular_distance_sum(candidate):
        # Shortest distance on [0, π)
        diff = np.abs(angles - candidate)
        diff = np.minimum(diff, np.pi - diff)
        return diff.sum()
    
    result = minimize_scalar(circular_distance_sum, bounds=(0, np.pi), method='bounded')
    return result.x

sols_standardized = copy.deepcopy(sols_controller)
for i, sol in enumerate(sols_controller):
    if sol is None:
        continue
    angles = sol.xyt[0,:,2].get() % np.pi
    ##print(i)
    median_angle = circular_median_mod_pi(angles)    
    #print(median_angle)
    if median_angle<np.pi/4 or median_angle>3*np.pi/4:
        # rotate 90 degrees
        sol.rotate(cp.array([np.pi/2], dtype=kgs.dtype_cp))
    angles = sol.xyt[0,:,2].get() % np.pi
    median_angle = circular_median_mod_pi(angles)   
    #print(median_angle)
    if median_angle>np.pi/2:
        # mirror
        sol.xyt[:,:,0]*=-1
        sol.xyt[:,:,2]*=-1
    angles = sol.xyt[0,:,2].get() % np.pi
    median_angle = circular_median_mod_pi(angles) 
    #print(median_angle)
    #plt.figure()
    #angles = sol.xyt[0,:,2].get() % np.pi
    #plt.hist(angles, bins=10)
    #median_angle = circular_median_mod_pi(angles)    
    #pack_vis_sol.pack_vis_sol(sol)
    #plt.title(f'N_trees={1+i}, median angle={median_angle*180/np.pi:.2f} deg')
    assert median_angle>=60*np.pi/180 and median_angle<=80*np.pi/180
    print(median_angle*180/np.pi)

61.466671684666096
70.0112208383682
63.54286911374217
67.23880317200123
67.0183844578964
72.33032952047724
61.02405989882475
66.59578560822227
72.47368932569378
65.09243967543402
62.93588837217167
64.08923598265947
66.4024636968035
72.31114428831317
65.4147553839374
71.6575307391155
70.28757888618124
66.06264154686093
65.02893977812676
66.095569448212
65.54909672930775
71.77596478138872
72.35374526660362
72.08941964811763
72.69452258596527
71.6606678476246
67.8677680462407
67.64369832860488
71.13312974617264
64.40658843990073
67.84669362706794
66.0681523303225
71.57163435412286
65.6456162547564
72.48261653612744
70.30374958682779
71.78888186781928
69.94677321017333
66.72631918653471
66.13771543124182
65.76148530538478
72.88365820804883
71.10485206951905
72.38684296264536
70.84570832163183
70.94375375140471
70.3179828518706
68.1445031874306
69.17754895315247
73.00271874390418
72.49844535245303
72.33689436857678
71.92214950925478
71.3192638111491
71.77311955696108
69.19621684004744
68.55

In [4]:
kgs.dill_save(kgs.code_dir + '../res/sols_standardized.pickle', sols_standardized)